In [2]:
using JuMP
using Cbc
using DataFrames
using Gurobi
using CSV

In [3]:
# m = Model(solver = CbcSolver())
m = Model(solver=GurobiSolver(Presolve=0))

data = CSV.read("supermarket_data.csv")
simple_dist = CSV.read("distmatrix_simple.csv")

v = data[:,2]

simple_dist = simple_dist[:,2:103]
c = simple_dist
    
@variables m begin
    y[1:102], Bin #1 if item i is taken
end

@variables m begin
    x[1:102,1:102], Bin #1 if direct path betweeen i and j
end

@variables m begin
    z[1:102]
end

@variables m begin
    t[1:102,1:102]
end

@constraint(m, y[1] == 1)
@constraint(m, y[102] == 1)
@constraint(m, z[1] == 0)
@constraint(m, sum(y[i] for i in 2:101) <= 15)
#@constraint(m, sum(x[i,1] for i in 1:102) == 0)
#@constraint(m, sum(x[102,j] for j in 1:102) == 0)

for i in 1:101
    @constraint(m, sum(x[i,j] for j in 2:102) == y[i])
end

for j in 2:102
    @constraint(m, sum(x[i,j] for i in 1:101) == y[j])
end

for j in 2:102
    @constraint(m, sum(t[i,j] for i in 1:102) == z[j])
end

for j in 1:101
    @constraint(m, sum(t[j,k] for k in 1:102) == (z[j] + sum(c[j,k]x[j,k] for k in 1:102)))
end

for j in 1:102
    for k in 2:102
        @constraint(m, t[j,k] >= x[j,k])
        @constraint(m, t[j,k] <= 90x[j,k])
    end
end

@objective(m, Max, sum(v[i]*y[i] for i in 1:102))


solve(m)

#for i in 2:102
#    println(i, getvalue(y[i]))
#end

# for i in 2:101
#     if getvalue(y[i]) == 1
#         println(i, " ", getvalue(y[i]))
#     end
# end

totdist = 0

# for i in 1:102
#     for j in 1:102
#         if getvalue(x[i,j]) == 1
#             println("i: ", i, ", j: ", j, )
#             totdist += c[i,j]
#         end
#     end
# end

# for i in 2:101
#     println(i, " ", getvalue(z[i]))
    
# end

for i in 1:102
    for j in 1:102
        if getvalue(t[i,j]) > 0
            println("i: ", i, ", j: ", j, "t: ", t[i,j])
        end
    end
end

# for j in 1:101
#     for k in 2:102
#         if getvalue(x[j,k]) > 0
#             println(getvalue(x[j,k]), " j: ", j, " k: ", k)
#         end
#         #println(c[j,k])
#         #print(sum(c[j,k]*x[j,k] for k in 1:102))
#     end
# end

println(totdist)

Academic license - for non-commercial use only
Optimize a model with 21012 rows, 21012 columns and 93022 nonzeros
Variable types: 10506 continuous, 10506 integer (10506 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [9e-01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 2.4900000
Variable types: 10505 continuous, 10507 integer (10506 binary)

Root relaxation: objective 1.629900e+02, 10804 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     162.9900000  162.99000  0.00%     -    0s

Explored 0 nodes (10804 simplex iterations) in 0.13 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 162.99 2.49 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.629900000000e+02, best bound 1.629900000000e+02, gap